# Demo of DeetForest VS XGBoost

GcForesr Link http://lamda.nju.edu.cn/code_gcForestCS.ashx

In [1]:
PATH_TO_GCFOREST_FOLDER="/home/yaantok1/Jira-1129 RS/"

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.pipeline import make_pipeline,Pipeline
from sklearn.pipeline import FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from category_encoders.cat_boost import CatBoostEncoder
from category_encoders.one_hot import OneHotEncoder
from sklearn.metrics import roc_auc_score
import statsmodels.stats.api as sms
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

#Imports for gcForestCs
import argparse
import sys
import pickle
sys.path.insert(0, PATH_TO_GCFOREST_FOLDER+"gcForestCS/gcForestCS/lib")

from gcforest.gcforestCS import GCForestCS
from gcforest.gcforest import GCForest
from gcforest.utils.config_utils import load_json
from gcforest.utils.log_utils import get_logger
import scipy.io as sio

In [3]:
#Confir from fcForestCS Example
def get_config():
    config = {}
    ca_config = {}
    ca_config["random_state"] = 0
    ca_config["max_layers"] = 200
    ca_config["early_stopping_rounds"] = 4
    ca_config["n_classes"] = 2
    ca_config["estimators"] = []
    ca_config["estimators"].append({"n_folds": 3, "type": "RandomForestClassifier", "n_estimators": 500, "max_depth": None, "n_jobs": -1})
    ca_config["estimators"].append({"n_folds": 3, "type": "ExtraTreesClassifier", "n_estimators": 500, "max_depth": None, "n_jobs": -1})
    config["cascadeCS"] = ca_config
    return config

In [4]:
class FeaturesExtractor(BaseEstimator, TransformerMixin):
    def __init__(self, features):
        self.features=features
    
    def fit(self, x, y=None):
        return self

    def transform(self, df):
        return df.loc[:, map(str.lower, self.features)]

# Load and Preprocess Data

You can find this dataset at https://archive.ics.uci.edu/ml/datasets/adult

In [5]:
columns=['age','workclass','fnlwgt','education','education-num','marital-status','occupation','relationship','race','sex','capital-gain','capital-loss','hours-per-week','native-country','Salary']

In [6]:
cathegorial_features=['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country']
numerical_features=['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']

In [8]:
categorical = Pipeline([
    ('extractor', FeaturesExtractor(cathegorial_features)), 
    ('coder', CatBoostEncoder())])

numerical = Pipeline([
    ('exractor', FeaturesExtractor(numerical_features))])

feature_union = FeatureUnion([
    ('numerical', numerical), 
    ('categorical', categorical)
])

pl_transform = make_pipeline(
    feature_union
)

In [9]:
df=pd.read_csv('adult.data',names=columns,index_col=False)
X=df[df.columns[:-1]]
Y=df['Salary']
Y=Y.apply(lambda i: 0 if i ==' <=50K' else 1)
x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size=0.2, random_state=42, stratify=Y)
x_train=x_train.reset_index(drop=True)
y_train=y_train.reset_index(drop=True)
x_test=x_test.reset_index(drop=True)
y_test=y_test.reset_index(drop=True)

In [10]:
pl_transform.fit(x_train,y_train)
x_train_transformed=pl_transform.transform(x_train)
x_test_transformed=pl_transform.transform(x_test)

## XGBoost

In [11]:
xg = xgb.XGBClassifier()
xg.fit(x_train_transformed,y_train);

In [12]:
roc_auc_score(y_test,xg.predict_proba(x_test_transformed)[:,1])

0.9256373681929799

In [64]:
roc_auc_scores=[]

kf = KFold(n_splits=5,random_state=1)

In [65]:
for train,test in kf.split(x_train):
    xg.fit(x_train_transformed[train,:],y_train.loc[train]);
    roc_auc_scores.append(roc_auc_score(y_train.loc[test],xg.predict_proba(x_train_transformed[test,:])[:,1]))

In [77]:
for train,test in KFold(n_splits=5,random_state=0).split(x_train):
    xg.fit(x_train_transformed[train,:],y_train.loc[train]);
    roc_auc_scores.append(roc_auc_score(y_train.loc[test],xg.predict_proba(x_train_transformed[test,:])[:,1]))

In [90]:
for train,test in KFold(n_splits=5,random_state=2).split(x_train):
    xg.fit(x_train_transformed[train,:],y_train.loc[train]);
    roc_auc_scores.append(roc_auc_score(y_train.loc[test],xg.predict_proba(x_train_transformed[test,:])[:,1]))

## gcForestCs

In [72]:
gcCS = GCForestCS(get_config())

In [73]:
roc_auc_scores_gcForest=[]

kf = KFold(n_splits=5,random_state=1)
i=0
for train,test in kf.split(x_train):
    gcCS.fit_transform(x_train_transformed[train,:],y_train.loc[train].values,
                      x_train_transformed[test,:],y_train.loc[test].values);
    roc_auc_scores_gcForest.append(roc_auc_score(y_train.loc[test],gcCS.predict_proba(x_train_transformed[test,:])[:,1]))
    print roc_auc_scores_gcForest

[ 2019-06-24 11:57:30,591][cascade_classifier_cs.fit_transform] X_groups_train.shape=[(20838, 14)],y_train.shape=(20838,),X_groups_test.shape=[(5210, 14)],y_test.shape=(5210,)
[ 2019-06-24 11:57:30,594][cascade_classifier_cs.fit_transform] group_dims=[14]
[ 2019-06-24 11:57:30,595][cascade_classifier_cs.fit_transform] group_starts=[0]
[ 2019-06-24 11:57:30,596][cascade_classifier_cs.fit_transform] group_ends=[14]
[ 2019-06-24 11:57:30,597][cascade_classifier_cs.fit_transform] X_train.shape=(20838, 14),X_test.shape=(5210, 14)
[ 2019-06-24 11:57:30,601][cascade_classifier_cs.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(20838, 14), X_cur_test.shape=(5210, 14)
[ 2019-06-24 11:57:30,603][cascade_classifier_cs.fit_transform] [layer=0] train_decay_level=1.0, # n_estimators before=500, # n_estimators now=500
[ 2019-06-24 11:57:33,981][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 3_folds.train_0.predict)=85.49%
[ 2019-06-24 11:57:37,289][kfold_wrapper.log_eva

[ 2019-06-24 11:59:07,295][kfold_wrapper.log_eval_metrics] Accuracy(layer_2 - estimator_1 - 3_folds.train_cv.predict)=64.23%
[ 2019-06-24 11:59:07,297][kfold_wrapper.log_eval_metrics] Accuracy(layer_2 - estimator_1 - 3_folds.test.predict)=66.14%
[ 2019-06-24 11:59:07,299][cascade_classifier_cs.calc_accuracy] Accuracy(layer_2 - train part2 accuracy)=65.41%
[ 2019-06-24 11:59:07,300][cascade_classifier_cs.fit_transform] ------------------------layer_2 - train accuracy 86.1359055572
[ 2019-06-24 11:59:07,301][cascade_classifier_cs.calc_accuracy] Accuracy(layer_2 - test part2 accuracy)=66.08%
[ 2019-06-24 11:59:07,302][cascade_classifier_cs.fit_transform] ------------------------layer_2 - test accuracy 85.7965451056
[ 2019-06-24 11:59:07,304][cascade_classifier_cs.getAutoThresh] Training accuracy now is 0.654138992386 and accuracy part1 needs to get 0.884712997462 (3 times)
[ 2019-06-24 11:59:07,310][cascade_classifier_cs.decide_thresh] #instances = 6173, num_thresh 6034, CX threshold: 0.8

[ 2019-06-24 12:00:37,283][kfold_wrapper.log_eval_metrics] Accuracy(layer_5 - estimator_0 - 3_folds.train_0.predict)=63.59%
[ 2019-06-24 12:00:44,743][kfold_wrapper.log_eval_metrics] Accuracy(layer_5 - estimator_0 - 3_folds.train_1.predict)=63.33%
[ 2019-06-24 12:00:52,524][kfold_wrapper.log_eval_metrics] Accuracy(layer_5 - estimator_0 - 3_folds.train_2.predict)=62.32%
[ 2019-06-24 12:00:53,407][kfold_wrapper.log_eval_metrics] Accuracy(layer_5 - estimator_0 - 3_folds.train_cv.predict)=63.08%
[ 2019-06-24 12:00:53,410][kfold_wrapper.log_eval_metrics] Accuracy(layer_5 - estimator_0 - 3_folds.test.predict)=66.35%
[ 2019-06-24 12:00:53,411][cascade_classifier_cs.fit_transform] [layer=5] train_decay_level=0.364464645634, # n_estimators before=500, # n_estimators now=1371
[ 2019-06-24 12:00:59,937][kfold_wrapper.log_eval_metrics] Accuracy(layer_5 - estimator_1 - 3_folds.train_0.predict)=64.50%
[ 2019-06-24 12:01:07,044][kfold_wrapper.log_eval_metrics] Accuracy(layer_5 - estimator_1 - 3_folds

[ 2019-06-24 12:02:44,620][cascade_classifier_cs.transform] X_groups_test.shape=[(5210, 14)]
[ 2019-06-24 12:02:44,622][cascade_classifier_cs.transform] group_dims=[14]
[ 2019-06-24 12:02:44,623][cascade_classifier_cs.transform] X_test.shape=(5210, 14)
[ 2019-06-24 12:02:44,624][cascade_classifier_cs.transform] [layer=0], #instances=5210
[ 2019-06-24 12:02:44,625][cascade_classifier_cs.transform] [layer=0] look_indexs=[0], X_cur_test.shape=(5210, 14)
[ 2019-06-24 12:02:47,391][cascade_classifier_cs.transform] [layer=1], #instances=1813
[ 2019-06-24 12:02:47,394][cascade_classifier_cs.transform] [layer=1] look_indexs=[0], X_cur_test.shape=(1813, 18)
[ 2019-06-24 12:02:51,962][cascade_classifier_cs.transform] [layer=2], #instances=1639
[ 2019-06-24 12:02:51,966][cascade_classifier_cs.transform] [layer=2] look_indexs=[0], X_cur_test.shape=(1639, 18)
[ 2019-06-24 12:02:56,810][cascade_classifier_cs.transform] [layer=3], #instances=1608
[ 2019-06-24 12:02:56,813][cascade_classifier_cs.trans

[0.9110249127078356]


...

[0.9110249127078356, 0.9096770594625583, 0.923903337042685, 0.9080834264338525]


[ 2019-06-24 12:19:56,975][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 3_folds.train_0.predict)=85.73%
[ 2019-06-24 12:20:00,575][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 3_folds.train_1.predict)=86.01%
[ 2019-06-24 12:20:03,943][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 3_folds.train_2.predict)=85.85%
[ 2019-06-24 12:20:04,306][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 3_folds.train_cv.predict)=85.86%
[ 2019-06-24 12:20:04,308][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 3_folds.test.predict)=85.56%
[ 2019-06-24 12:20:04,327][cascade_classifier_cs.fit_transform] [layer=0] train_decay_level=1.0, # n_estimators before=500, # n_estimators now=500
[ 2019-06-24 12:20:07,168][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_1 - 3_folds.train_0.predict)=84.47%
[ 2019-06-24 12:20:10,404][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_1 - 3_folds.train_1.pre

[ 2019-06-24 12:21:32,518][cascade_classifier_cs.getAutoThresh] Training accuracy now is 0.660448993067 and accuracy part1 needs to get 0.886816331022 (3 times)
[ 2019-06-24 12:21:32,524][cascade_classifier_cs.decide_thresh] #instances = 6058, num_thresh 6012, CX threshold: 0.913871943951
[ 2019-06-24 12:21:32,526][cascade_classifier_cs.confidence_screening] In layer 3, train num is 6034, part decay is 0.977482585453
[ 2019-06-24 12:21:32,527][cascade_classifier_cs.calc_accuracy] Accuracy(layer_2 - train.1part(46))=89.13%
[ 2019-06-24 12:21:32,529][cascade_classifier_cs.calc_accuracy] Accuracy(layer_2 - train.2part(6012))=65.87%
[ 2019-06-24 12:21:32,530][cascade_classifier_cs.confidence_screening_test] In layer 3, test num is 1608, part decay is 0.981086028066
[ 2019-06-24 12:21:32,531][cascade_classifier_cs.calc_accuracy] Accuracy(layer_2 - test.1part(6))=83.33%
[ 2019-06-24 12:21:32,532][cascade_classifier_cs.calc_accuracy] Accuracy(layer_2 - test.2part(1590))=66.73%
[ 2019-06-24 12

[ 2019-06-24 12:23:15,885][cascade_classifier_cs.fit_transform] [layer=5] train_decay_level=0.364464645634, # n_estimators before=500, # n_estimators now=1371
[ 2019-06-24 12:23:21,803][kfold_wrapper.log_eval_metrics] Accuracy(layer_5 - estimator_1 - 3_folds.train_0.predict)=62.49%
[ 2019-06-24 12:23:28,508][kfold_wrapper.log_eval_metrics] Accuracy(layer_5 - estimator_1 - 3_folds.train_1.predict)=63.13%
[ 2019-06-24 12:23:35,471][kfold_wrapper.log_eval_metrics] Accuracy(layer_5 - estimator_1 - 3_folds.train_2.predict)=63.83%
[ 2019-06-24 12:23:36,335][kfold_wrapper.log_eval_metrics] Accuracy(layer_5 - estimator_1 - 3_folds.train_cv.predict)=63.15%
[ 2019-06-24 12:23:36,338][kfold_wrapper.log_eval_metrics] Accuracy(layer_5 - estimator_1 - 3_folds.test.predict)=66.96%
[ 2019-06-24 12:23:36,339][cascade_classifier_cs.calc_accuracy] Accuracy(layer_5 - train part2 accuracy)=64.62%
[ 2019-06-24 12:23:36,340][cascade_classifier_cs.fit_transform] ------------------------layer_5 - train accurac

[0.9110249127078356, 0.9096770594625583, 0.923903337042685, 0.9080834264338525, 0.9078877362565961]


In [76]:
kf = KFold(n_splits=5,random_state=0)
i=0
for train,test in kf.split(x_train):
    gcCS.fit_transform(x_train_transformed[train,:],y_train.loc[train].values,
                      x_train_transformed[test,:],y_train.loc[test].values);
    roc_auc_scores_gcForest.append(roc_auc_score(y_train.loc[test],gcCS.predict_proba(x_train_transformed[test,:])[:,1]))
    print roc_auc_scores_gcForest

[ 2019-06-24 12:40:27,756][cascade_classifier_cs.fit_transform] X_groups_train.shape=[(20838, 14)],y_train.shape=(20838,),X_groups_test.shape=[(5210, 14)],y_test.shape=(5210,)
[ 2019-06-24 12:40:27,759][cascade_classifier_cs.fit_transform] group_dims=[14]
[ 2019-06-24 12:40:27,760][cascade_classifier_cs.fit_transform] group_starts=[0]
[ 2019-06-24 12:40:27,761][cascade_classifier_cs.fit_transform] group_ends=[14]
[ 2019-06-24 12:40:27,761][cascade_classifier_cs.fit_transform] X_train.shape=(20838, 14),X_test.shape=(5210, 14)
[ 2019-06-24 12:40:27,765][cascade_classifier_cs.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(20838, 14), X_cur_test.shape=(5210, 14)
[ 2019-06-24 12:40:27,766][cascade_classifier_cs.fit_transform] [layer=0] train_decay_level=1.0, # n_estimators before=500, # n_estimators now=500
[ 2019-06-24 12:40:30,854][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 3_folds.train_0.predict)=85.49%
[ 2019-06-24 12:40:34,095][kfold_wrapper.log_eva

[ 2019-06-24 12:42:09,584][kfold_wrapper.log_eval_metrics] Accuracy(layer_2 - estimator_1 - 3_folds.train_cv.predict)=64.23%
[ 2019-06-24 12:42:09,586][kfold_wrapper.log_eval_metrics] Accuracy(layer_2 - estimator_1 - 3_folds.test.predict)=66.14%
[ 2019-06-24 12:42:09,612][cascade_classifier_cs.calc_accuracy] Accuracy(layer_2 - train part2 accuracy)=65.41%
[ 2019-06-24 12:42:09,613][cascade_classifier_cs.fit_transform] ------------------------layer_2 - train accuracy 86.1359055572
[ 2019-06-24 12:42:09,614][cascade_classifier_cs.calc_accuracy] Accuracy(layer_2 - test part2 accuracy)=66.08%
[ 2019-06-24 12:42:09,615][cascade_classifier_cs.fit_transform] ------------------------layer_2 - test accuracy 85.7965451056
[ 2019-06-24 12:42:09,616][cascade_classifier_cs.getAutoThresh] Training accuracy now is 0.654138992386 and accuracy part1 needs to get 0.884712997462 (3 times)
[ 2019-06-24 12:42:09,622][cascade_classifier_cs.decide_thresh] #instances = 6173, num_thresh 6034, CX threshold: 0.8

[ 2019-06-24 12:43:42,132][kfold_wrapper.log_eval_metrics] Accuracy(layer_5 - estimator_0 - 3_folds.train_0.predict)=63.59%
[ 2019-06-24 12:43:49,588][kfold_wrapper.log_eval_metrics] Accuracy(layer_5 - estimator_0 - 3_folds.train_1.predict)=63.33%
[ 2019-06-24 12:43:56,726][kfold_wrapper.log_eval_metrics] Accuracy(layer_5 - estimator_0 - 3_folds.train_2.predict)=62.32%
[ 2019-06-24 12:43:57,610][kfold_wrapper.log_eval_metrics] Accuracy(layer_5 - estimator_0 - 3_folds.train_cv.predict)=63.08%
[ 2019-06-24 12:43:57,612][kfold_wrapper.log_eval_metrics] Accuracy(layer_5 - estimator_0 - 3_folds.test.predict)=66.35%
[ 2019-06-24 12:43:57,617][cascade_classifier_cs.fit_transform] [layer=5] train_decay_level=0.364464645634, # n_estimators before=500, # n_estimators now=1371
[ 2019-06-24 12:44:03,728][kfold_wrapper.log_eval_metrics] Accuracy(layer_5 - estimator_1 - 3_folds.train_0.predict)=64.50%
[ 2019-06-24 12:44:10,668][kfold_wrapper.log_eval_metrics] Accuracy(layer_5 - estimator_1 - 3_folds

[ 2019-06-24 12:45:45,623][cascade_classifier_cs.transform] X_groups_test.shape=[(5210, 14)]
[ 2019-06-24 12:45:45,625][cascade_classifier_cs.transform] group_dims=[14]
[ 2019-06-24 12:45:45,625][cascade_classifier_cs.transform] X_test.shape=(5210, 14)
[ 2019-06-24 12:45:45,626][cascade_classifier_cs.transform] [layer=0], #instances=5210
[ 2019-06-24 12:45:45,628][cascade_classifier_cs.transform] [layer=0] look_indexs=[0], X_cur_test.shape=(5210, 14)
[ 2019-06-24 12:45:48,276][cascade_classifier_cs.transform] [layer=1], #instances=1813
[ 2019-06-24 12:45:48,279][cascade_classifier_cs.transform] [layer=1] look_indexs=[0], X_cur_test.shape=(1813, 18)
[ 2019-06-24 12:45:52,721][cascade_classifier_cs.transform] [layer=2], #instances=1639
[ 2019-06-24 12:45:52,723][cascade_classifier_cs.transform] [layer=2] look_indexs=[0], X_cur_test.shape=(1639, 18)
[ 2019-06-24 12:45:57,568][cascade_classifier_cs.transform] [layer=3], #instances=1608
[ 2019-06-24 12:45:57,570][cascade_classifier_cs.trans

[0.9110249127078356, 0.9096770594625583, 0.923903337042685, 0.9080834264338525, 0.9078877362565961, 0.9110249127078356]


...

[0.9110249127078356, 0.9096770594625583, 0.923903337042685, 0.9080834264338525, 0.9078877362565961, 0.9110249127078356, 0.9096770594625583, 0.923903337042685, 0.9080834264338525]


[ 2019-06-24 13:02:52,439][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 3_folds.train_0.predict)=85.73%
[ 2019-06-24 13:02:55,782][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 3_folds.train_1.predict)=86.01%
[ 2019-06-24 13:02:59,366][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 3_folds.train_2.predict)=85.85%
[ 2019-06-24 13:02:59,820][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 3_folds.train_cv.predict)=85.86%
[ 2019-06-24 13:02:59,821][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 3_folds.test.predict)=85.56%
[ 2019-06-24 13:02:59,837][cascade_classifier_cs.fit_transform] [layer=0] train_decay_level=1.0, # n_estimators before=500, # n_estimators now=500
[ 2019-06-24 13:03:02,724][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_1 - 3_folds.train_0.predict)=84.47%
[ 2019-06-24 13:03:06,064][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_1 - 3_folds.train_1.pre

[ 2019-06-24 13:04:30,704][cascade_classifier_cs.getAutoThresh] Training accuracy now is 0.660448993067 and accuracy part1 needs to get 0.886816331022 (3 times)
[ 2019-06-24 13:04:30,715][cascade_classifier_cs.decide_thresh] #instances = 6058, num_thresh 6012, CX threshold: 0.913871943951
[ 2019-06-24 13:04:30,719][cascade_classifier_cs.confidence_screening] In layer 3, train num is 6034, part decay is 0.977482585453
[ 2019-06-24 13:04:30,720][cascade_classifier_cs.calc_accuracy] Accuracy(layer_2 - train.1part(46))=89.13%
[ 2019-06-24 13:04:30,721][cascade_classifier_cs.calc_accuracy] Accuracy(layer_2 - train.2part(6012))=65.87%
[ 2019-06-24 13:04:30,724][cascade_classifier_cs.confidence_screening_test] In layer 3, test num is 1608, part decay is 0.981086028066
[ 2019-06-24 13:04:30,725][cascade_classifier_cs.calc_accuracy] Accuracy(layer_2 - test.1part(6))=83.33%
[ 2019-06-24 13:04:30,726][cascade_classifier_cs.calc_accuracy] Accuracy(layer_2 - test.2part(1590))=66.73%
[ 2019-06-24 13

[ 2019-06-24 13:06:20,088][cascade_classifier_cs.fit_transform] [layer=5] train_decay_level=0.364464645634, # n_estimators before=500, # n_estimators now=1371
[ 2019-06-24 13:06:26,164][kfold_wrapper.log_eval_metrics] Accuracy(layer_5 - estimator_1 - 3_folds.train_0.predict)=62.49%
[ 2019-06-24 13:06:33,040][kfold_wrapper.log_eval_metrics] Accuracy(layer_5 - estimator_1 - 3_folds.train_1.predict)=63.13%
[ 2019-06-24 13:06:40,523][kfold_wrapper.log_eval_metrics] Accuracy(layer_5 - estimator_1 - 3_folds.train_2.predict)=63.83%
[ 2019-06-24 13:06:41,432][kfold_wrapper.log_eval_metrics] Accuracy(layer_5 - estimator_1 - 3_folds.train_cv.predict)=63.15%
[ 2019-06-24 13:06:41,434][kfold_wrapper.log_eval_metrics] Accuracy(layer_5 - estimator_1 - 3_folds.test.predict)=66.96%
[ 2019-06-24 13:06:41,435][cascade_classifier_cs.calc_accuracy] Accuracy(layer_5 - train part2 accuracy)=64.62%
[ 2019-06-24 13:06:41,436][cascade_classifier_cs.fit_transform] ------------------------layer_5 - train accurac

[0.9110249127078356, 0.9096770594625583, 0.923903337042685, 0.9080834264338525, 0.9078877362565961, 0.9110249127078356, 0.9096770594625583, 0.923903337042685, 0.9080834264338525, 0.9078877362565961]


In [88]:
kf = KFold(n_splits=5,random_state=2)
i=0
for train,test in kf.split(x_train):
    gcCS.fit_transform(x_train_transformed[train,:],y_train.loc[train].values,
                      x_train_transformed[test,:],y_train.loc[test].values);
    roc_auc_scores_gcForest.append(roc_auc_score(y_train.loc[test],gcCS.predict_proba(x_train_transformed[test,:])[:,1]))
    print roc_auc_scores_gcForest

[ 2019-06-24 16:12:53,022][cascade_classifier_cs.fit_transform] X_groups_train.shape=[(20838, 14)],y_train.shape=(20838,),X_groups_test.shape=[(5210, 14)],y_test.shape=(5210,)
[ 2019-06-24 16:12:53,025][cascade_classifier_cs.fit_transform] group_dims=[14]
[ 2019-06-24 16:12:53,026][cascade_classifier_cs.fit_transform] group_starts=[0]
[ 2019-06-24 16:12:53,027][cascade_classifier_cs.fit_transform] group_ends=[14]
[ 2019-06-24 16:12:53,028][cascade_classifier_cs.fit_transform] X_train.shape=(20838, 14),X_test.shape=(5210, 14)
[ 2019-06-24 16:12:53,031][cascade_classifier_cs.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(20838, 14), X_cur_test.shape=(5210, 14)
[ 2019-06-24 16:12:53,032][cascade_classifier_cs.fit_transform] [layer=0] train_decay_level=1.0, # n_estimators before=500, # n_estimators now=500
[ 2019-06-24 16:12:56,411][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 3_folds.train_0.predict)=85.49%
[ 2019-06-24 16:12:59,975][kfold_wrapper.log_eva

[ 2019-06-24 16:14:31,064][kfold_wrapper.log_eval_metrics] Accuracy(layer_2 - estimator_1 - 3_folds.train_cv.predict)=64.23%
[ 2019-06-24 16:14:31,066][kfold_wrapper.log_eval_metrics] Accuracy(layer_2 - estimator_1 - 3_folds.test.predict)=66.14%
[ 2019-06-24 16:14:31,098][cascade_classifier_cs.calc_accuracy] Accuracy(layer_2 - train part2 accuracy)=65.41%
[ 2019-06-24 16:14:31,100][cascade_classifier_cs.fit_transform] ------------------------layer_2 - train accuracy 86.1359055572
[ 2019-06-24 16:14:31,101][cascade_classifier_cs.calc_accuracy] Accuracy(layer_2 - test part2 accuracy)=66.08%
[ 2019-06-24 16:14:31,102][cascade_classifier_cs.fit_transform] ------------------------layer_2 - test accuracy 85.7965451056
[ 2019-06-24 16:14:31,103][cascade_classifier_cs.getAutoThresh] Training accuracy now is 0.654138992386 and accuracy part1 needs to get 0.884712997462 (3 times)
[ 2019-06-24 16:14:31,109][cascade_classifier_cs.decide_thresh] #instances = 6173, num_thresh 6034, CX threshold: 0.8

[ 2019-06-24 16:16:00,172][kfold_wrapper.log_eval_metrics] Accuracy(layer_5 - estimator_0 - 3_folds.train_0.predict)=63.59%
[ 2019-06-24 16:16:07,101][kfold_wrapper.log_eval_metrics] Accuracy(layer_5 - estimator_0 - 3_folds.train_1.predict)=63.33%
[ 2019-06-24 16:16:13,909][kfold_wrapper.log_eval_metrics] Accuracy(layer_5 - estimator_0 - 3_folds.train_2.predict)=62.32%
[ 2019-06-24 16:16:14,558][kfold_wrapper.log_eval_metrics] Accuracy(layer_5 - estimator_0 - 3_folds.train_cv.predict)=63.08%
[ 2019-06-24 16:16:14,560][kfold_wrapper.log_eval_metrics] Accuracy(layer_5 - estimator_0 - 3_folds.test.predict)=66.35%
[ 2019-06-24 16:16:14,561][cascade_classifier_cs.fit_transform] [layer=5] train_decay_level=0.364464645634, # n_estimators before=500, # n_estimators now=1371
[ 2019-06-24 16:16:20,469][kfold_wrapper.log_eval_metrics] Accuracy(layer_5 - estimator_1 - 3_folds.train_0.predict)=64.50%
[ 2019-06-24 16:16:27,127][kfold_wrapper.log_eval_metrics] Accuracy(layer_5 - estimator_1 - 3_folds

[ 2019-06-24 16:18:00,182][cascade_classifier_cs.transform] X_groups_test.shape=[(5210, 14)]
[ 2019-06-24 16:18:00,184][cascade_classifier_cs.transform] group_dims=[14]
[ 2019-06-24 16:18:00,185][cascade_classifier_cs.transform] X_test.shape=(5210, 14)
[ 2019-06-24 16:18:00,186][cascade_classifier_cs.transform] [layer=0], #instances=5210
[ 2019-06-24 16:18:00,188][cascade_classifier_cs.transform] [layer=0] look_indexs=[0], X_cur_test.shape=(5210, 14)
[ 2019-06-24 16:18:02,515][cascade_classifier_cs.transform] [layer=1], #instances=1813
[ 2019-06-24 16:18:02,518][cascade_classifier_cs.transform] [layer=1] look_indexs=[0], X_cur_test.shape=(1813, 18)
[ 2019-06-24 16:18:06,655][cascade_classifier_cs.transform] [layer=2], #instances=1639
[ 2019-06-24 16:18:06,658][cascade_classifier_cs.transform] [layer=2] look_indexs=[0], X_cur_test.shape=(1639, 18)
[ 2019-06-24 16:18:10,994][cascade_classifier_cs.transform] [layer=3], #instances=1608
[ 2019-06-24 16:18:10,997][cascade_classifier_cs.trans

[0.9110249127078356, 0.9096770594625583, 0.923903337042685, 0.9080834264338525, 0.9078877362565961, 0.9110249127078356, 0.9096770594625583, 0.923903337042685, 0.9080834264338525, 0.9078877362565961, 0.9110249127078356]


...

[0.9110249127078356, 0.9096770594625583, 0.923903337042685, 0.9080834264338525, 0.9078877362565961, 0.9110249127078356, 0.9096770594625583, 0.923903337042685, 0.9080834264338525, 0.9078877362565961, 0.9110249127078356, 0.9096770594625583, 0.923903337042685, 0.9080834264338525]


[ 2019-06-24 16:34:17,055][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 3_folds.train_0.predict)=85.73%
[ 2019-06-24 16:34:20,608][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 3_folds.train_1.predict)=86.01%
[ 2019-06-24 16:34:24,107][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 3_folds.train_2.predict)=85.85%
[ 2019-06-24 16:34:24,566][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 3_folds.train_cv.predict)=85.86%
[ 2019-06-24 16:34:24,568][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 3_folds.test.predict)=85.56%
[ 2019-06-24 16:34:24,581][cascade_classifier_cs.fit_transform] [layer=0] train_decay_level=1.0, # n_estimators before=500, # n_estimators now=500
[ 2019-06-24 16:34:27,291][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_1 - 3_folds.train_0.predict)=84.47%
[ 2019-06-24 16:34:30,161][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_1 - 3_folds.train_1.pre

[ 2019-06-24 16:35:51,033][cascade_classifier_cs.getAutoThresh] Training accuracy now is 0.660448993067 and accuracy part1 needs to get 0.886816331022 (3 times)
[ 2019-06-24 16:35:51,038][cascade_classifier_cs.decide_thresh] #instances = 6058, num_thresh 6012, CX threshold: 0.913871943951
[ 2019-06-24 16:35:51,040][cascade_classifier_cs.confidence_screening] In layer 3, train num is 6034, part decay is 0.977482585453
[ 2019-06-24 16:35:51,041][cascade_classifier_cs.calc_accuracy] Accuracy(layer_2 - train.1part(46))=89.13%
[ 2019-06-24 16:35:51,042][cascade_classifier_cs.calc_accuracy] Accuracy(layer_2 - train.2part(6012))=65.87%
[ 2019-06-24 16:35:51,044][cascade_classifier_cs.confidence_screening_test] In layer 3, test num is 1608, part decay is 0.981086028066
[ 2019-06-24 16:35:51,045][cascade_classifier_cs.calc_accuracy] Accuracy(layer_2 - test.1part(6))=83.33%
[ 2019-06-24 16:35:51,046][cascade_classifier_cs.calc_accuracy] Accuracy(layer_2 - test.2part(1590))=66.73%
[ 2019-06-24 16

[ 2019-06-24 16:37:37,196][cascade_classifier_cs.fit_transform] [layer=5] train_decay_level=0.364464645634, # n_estimators before=500, # n_estimators now=1371
[ 2019-06-24 16:37:43,579][kfold_wrapper.log_eval_metrics] Accuracy(layer_5 - estimator_1 - 3_folds.train_0.predict)=62.49%
[ 2019-06-24 16:37:51,013][kfold_wrapper.log_eval_metrics] Accuracy(layer_5 - estimator_1 - 3_folds.train_1.predict)=63.13%
[ 2019-06-24 16:37:58,118][kfold_wrapper.log_eval_metrics] Accuracy(layer_5 - estimator_1 - 3_folds.train_2.predict)=63.83%
[ 2019-06-24 16:37:58,996][kfold_wrapper.log_eval_metrics] Accuracy(layer_5 - estimator_1 - 3_folds.train_cv.predict)=63.15%
[ 2019-06-24 16:37:58,998][kfold_wrapper.log_eval_metrics] Accuracy(layer_5 - estimator_1 - 3_folds.test.predict)=66.96%
[ 2019-06-24 16:37:59,000][cascade_classifier_cs.calc_accuracy] Accuracy(layer_5 - train part2 accuracy)=64.62%
[ 2019-06-24 16:37:59,000][cascade_classifier_cs.fit_transform] ------------------------layer_5 - train accurac

[0.9110249127078356, 0.9096770594625583, 0.923903337042685, 0.9080834264338525, 0.9078877362565961, 0.9110249127078356, 0.9096770594625583, 0.923903337042685, 0.9080834264338525, 0.9078877362565961, 0.9110249127078356, 0.9096770594625583, 0.923903337042685, 0.9080834264338525, 0.9078877362565961]


# Results

In [69]:
roc_auc_scores=[0.9214118036039015, 0.9220632940977193, 0.9277699827633614, 0.9170108247659934, 0.9173910672239793]

In [74]:
roc_auc_scores_gcForest=[0.9110249127078356, 0.9096770594625583, 0.923903337042685, 0.9080834264338525, 0.9078877362565961]

In [75]:
print sms.DescrStatsW(roc_auc_scores).tconfint_mean()
print sms.DescrStatsW(roc_auc_scores_gcForest).tconfint_mean() 

(0.9157175679711728, 0.9265412210108092)
(0.9037808686586117, 0.9204497201027995)


In [89]:
roc_auc_scores_gcForest_N_CV=[0.9110249127078356, 0.9096770594625583, 0.923903337042685, 0.9080834264338525, 0.9078877362565961, 0.9110249127078356, 0.9096770594625583, 0.923903337042685, 0.9080834264338525, 0.9078877362565961, 0.9110249127078356, 0.9096770594625583, 0.923903337042685, 0.9080834264338525, 0.9078877362565961]

In [91]:
print roc_auc_scores

[0.9214118036039015, 0.9220632940977193, 0.9277699827633614, 0.9170108247659934, 0.9173910672239793, 0.9214118036039015, 0.9220632940977193, 0.9277699827633614, 0.9170108247659934, 0.9173910672239793, 0.9214118036039015, 0.9220632940977193, 0.9277699827633614, 0.9170108247659934, 0.9173910672239793]


In [93]:
roc_auc_scores_N_CV=[0.9214118036039015, 0.9220632940977193, 0.9277699827633614, 0.9170108247659934, 0.9173910672239793, 0.9214118036039015, 0.9220632940977193, 0.9277699827633614, 0.9170108247659934, 0.9173910672239793, 0.9214118036039015, 0.9220632940977193, 0.9277699827633614, 0.9170108247659934, 0.9173910672239793]

In [94]:
print sms.DescrStatsW(roc_auc_scores_N_CV).tconfint_mean()
print sms.DescrStatsW(roc_auc_scores_gcForest_N_CV).tconfint_mean() 

(0.9188947684339486, 0.9233640205480336)
(0.9086738823507978, 0.9155567064106132)


Тест на нормальность распределения. $H_0\colon$  попарные разности распределены нормально.
$H_1\colon$  не нормально.

In [96]:
print "Shapiro-Wilk normality test, W-statistic: %f, p-value: %f" % stats.shapiro(np.array(roc_auc_scores_N_CV) - 
                                                                                  np.array(roc_auc_scores_gcForest_N_CV))

Shapiro-Wilk normality test, W-statistic: 0.821433, p-value: 0.007020


Критерий Стьюдента:
$H_0\colon$ средние значения одинаковы
$H_1\colon$ не одинаковы.

In [97]:
print "Rel. Test on Equal Means, W-statistic: %f, p-value: %f" % stats.ttest_rel(roc_auc_scores_N_CV, roc_auc_scores_gcForest_N_CV)

Rel. Test on Equal Means, W-statistic: 11.924683, p-value: 0.000000


In [98]:
print "95%% confidence interval: [%f, %f]" % DescrStatsW(np.array(roc_auc_scores_N_CV) - 
                                                                                  np.array(roc_auc_scores_gcForest_N_CV
                                                                                          )).tconfint_mean()

95% confidence interval: [0.007393, 0.010635]
